This workshop will work in a terminal. The first thing to do is to open a new terminal: from the jupyter home page click the new button on the top right and select terminal. You might want to open the terminal in a new window (right click on terminal and select open in new window). Don't close this notebook (until you have finished the tutorial)!

Lines in this notebook that begin with $ are the lines that you type into the terminal command line. Sample input files can be found in data/implicit and data/solvated. If you want to add a new cell (perhaps at the end), select "Insert" -> "Insert Cell Below" or "Insert Cell Above" from the menus above.

You will want to keep your workspace tidy, so we will organize files into directories. Start by making a directory for your current work

$ mkdir modelling

Enter that directory

$ cd modelling

You may want to make directories for gbsa (implicit) and solvated simulations (notebook 2) for later on.

$ mkdir gbsa

$ mkdir solvated

# Using tLeap

The tLeap module connects the protein structure with the AMBER forcefield.

AMBER contains “residue templates” for standard biological units (eg amino acids) that the program uses to assign forcefield parameters from a pdb file. It understands chemistry!

tLeap produces files called “parm” (or top) and “crd”.
“parm” contains the parameters and connectivities
“crd” contains the starting coordinates.

* Choose your forcefield

There are different forcefields even within AMBER!

1 - parm14SB (recommended for proteins)
source leaprc.ff14SB

2 - parmbsc1 (recommended for DNA)

3 - Gaff (generalised AMBER forcefield) – can simulate anything with this – caution needed!!
source leaprc.gaff


* Change to the directory where you want your implicit solvent simulations

$ cd gbsa

* Start the tLeap module:

$ tleap

* Load the forcefield.

To use the 14SB forcefield:

$ source leaprc.protein.ff14SB

The name of the forcefield file can depend on the version of AmberTools. The command above will work for Amber18, but for older versions of Amber you might need leaprc.ff14SB.

The AMBER manual and additional tutorials can be found at http://ambermd.org

# Building peptides in tLeap

To see which molecules you have available, type:

$ list

Think of 5 amino acids ~ eg GLY, ALA, TYR, PRO and ASP

Now type:

$ peptide = sequence {NGLY ALA TYR PRO CASP}

You need NXXX and CXXX at the ends of the peptide to correctly chemically cap the ends. You can use any 5 amino acids (the ones listed here are just an example).

# Save your molecule:

$ savepdb peptide YOUR_FILE_NAME.pdb

$ saveamberparm peptide YOUR_FILE_NAME.parm7 YOUR_FILE_NAME.crd

and exit tleap to return to the terminal shell:

$ quit

# Implicit solvent simulation of your peptide

To run Sander you need:
1. The parm (topology) and crd (coordinate) files
2. Sander input files (min1.in, min2.in, md1.in, md2.in, md3.in)

Take a look at the input files (implicit-\*.in). Try and get a gist of what the input file is doing. 

What is the difference between implicit-min1.in and implicit-min2.in? How much dynamics is involved in each step of the equilibration?

# Using sander

The molecular dynamics is run by the sander module. The sander executable is found in $AMBERHOME/bin. You can read the manual to find the details of all the options for sander. Some of the important ones are:

    -O or -A : Overwrite or Append output files if they already exist
    
    -i FILE_NAME.in (input) : control data for the run
    
    -o FILE_NAME.out (output) : user readable information
    
    -p FILE_NAME.parm7 (input) : molecular topology, force field, atom and residue names, periodic box type
    
    -c FILE_NAME.crd (input) : initial coordinates and periodic box size (may include velocities)
    
    -r FILE_NAME.rst (output) : final coordinates, velocities, and box dimentions (for restarting run) 
    
* Example (you will have to use your own filenames)

$ sander -O -i min1.in -o min1.out -inf min1.inf -c pep.crd -r pep.rst -p pep.parm7 -ref pep.crd -x pepmin1.nc

# Equilibration

To equilibrate the system, we first relax by running an energy minimisation (min1.in, min2.in). This helps to remove any bad contacts (slightly misplaced atoms) in the initial structure.

We then heat the system up in the presence of restraints on the solute (md1.in). Heating the system is followed by MD at the desired temperature (md2.in) and then removing the restraints (md3.in). This stepwise equilibration procedure allows the system to gradually relax without changing too much at any one time (which could cause simulations to become unstable and crash the MD program).

This equilibration needs to be particularly gentle for DNA simulations to ensure that the solvent and counterions screen the negatively charged backbone.

# Visualising the results

There are several programs available for visualising MD trajectories (VMD and Chimera are popular). In Jupyter notebooks, we can use NGLview (with MDtraj to import the trajectory).

In [ ]:
import mdtraj as mdt
import nglview as nv

In [ ]:
# select your files
top_file = 'YOUR_FILENAME.parm7'
traj_file = 'YOUR_FILENAME.nc'

In [ ]:
# load trajectory with MDtraj
traj = mdt.load(traj_file, top=top_file)

In [ ]:
#view with NGLview
view = nv.show_mdtraj(traj)

# Clear all representations to try new ones
view.clear_representations()

# Show licorice style representation
view.add_licorice()
view

# Repeat or replicate simulations

One way to perform an independent “repeat” of a simulation is to reassign the velocities at a chosen point in the trajectory. 

The ntx and irest flags in the .in file change (ntx=1, irest=0).

Make a new directory.

Running on from your restart file from md2 (eg pep.md2), run an independent repeat of your previous simulation by asking md3.in to reassign a new set of velocities. Call this trajectory something different.

Compare the two trajectories, and convince yourself that they are different.